In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import math
import missingno as msno

# Marlon's stuk

In [2]:
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print ("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")

In [3]:
tic()
df = pd.read_csv("small_data/50000_training_set_VU_DM.csv")
toc()

# Code to replace missing prop_review_score values with corresponding propstarrating vlaues.
def fill_prop_review_score(df):
    for i in range(len(df)):
        if math.isnan(df['prop_review_score'][i]):
            df['prop_review_score'][i] = df['prop_starrating'][i]
    return(df)
tic()
df = fill_prop_review_score(df)
toc()

Elapsed time is 0.2229146957397461 seconds.


<ipython-input-3-d966af75fcf3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prop_review_score'][i] = df['prop_starrating'][i]


Elapsed time is 0.3091726303100586 seconds.


In [4]:
# tic()
# # Drop column 'prop_location_score2', too many missing values.
# df = df.drop(columns=['prop_location_score2'])
# toc()

In [5]:
tic()
# Code which adds columns with the average rate,inventory, and percentage difference between Expedia and competitors.
def make_rates(data):
    data['comp_rate'] = data[['comp1_rate',
                            'comp2_rate',
                            'comp3_rate',
                            'comp4_rate',
                            'comp5_rate',
                            'comp6_rate',
                            'comp7_rate',
                            'comp8_rate']].mean(axis=1)
    data['comp_rate'] = data['comp_rate'].fillna(0)
    
    data['comp_inv'] = data[['comp1_inv',
                            'comp2_inv',
                            'comp3_inv',
                            'comp4_inv',
                            'comp5_inv',
                            'comp6_inv',
                            'comp7_inv',
                            'comp8_inv']].mean(axis=1)
    data['comp_inv'] = data['comp_inv'].fillna(0)
    
    data['comp_percent'] = data[['comp1_rate_percent_diff',
                            'comp2_rate_percent_diff',
                            'comp3_rate_percent_diff',
                            'comp4_rate_percent_diff',
                            'comp5_rate_percent_diff',
                            'comp6_rate_percent_diff',
                            'comp7_rate_percent_diff',
                            'comp8_rate_percent_diff']].mean(axis=1)
    data['comp_percent'] = data['comp_percent'].fillna(0)
    
    return(data)
df = make_rates(df)
toc()

Elapsed time is 0.06382393836975098 seconds.


In [6]:
tic()
# Dorp column 'prop_location_score2', too many missing values.
df = df.drop(columns=['comp1_rate',
                 'comp2_rate',
                 'comp3_rate',
                 'comp4_rate',
                 'comp5_rate',
                 'comp6_rate',
                 'comp7_rate',
                 'comp8_rate',
                 'comp1_inv',
                 'comp2_inv',
                 'comp3_inv',
                 'comp4_inv',
                 'comp5_inv',
                 'comp6_inv',
                 'comp7_inv',
                 'comp8_inv',
                 'comp1_rate_percent_diff',
                 'comp2_rate_percent_diff',
                 'comp3_rate_percent_diff',
                 'comp4_rate_percent_diff',
                 'comp5_rate_percent_diff',
                 'comp6_rate_percent_diff',
                 'comp7_rate_percent_diff',
                 'comp8_rate_percent_diff'])
toc()

Elapsed time is 0.028921842575073242 seconds.


In [7]:
for var in df:
    col = df[var]
    missing = col.isnull().values.sum()
    if missing >0:
        print('the number is missing values in', var, 'is', col.isnull().values.sum())


the number is missing values in visitor_hist_starrating is 47435
the number is missing values in visitor_hist_adr_usd is 47435
the number is missing values in prop_location_score2 is 10892
the number is missing values in srch_query_affinity_score is 46736
the number is missing values in orig_destination_distance is 15845
the number is missing values in gross_bookings_usd is 48623


# Daan's stuk

In [8]:
def orig_dest_distance_pandas_backup(visitor_country_id, prop_country_id, orig_destination_distance, i):
    orig = visitor_country_id[i]
    dest = prop_country_id[i]

    # check if someone has the exact same but with a distance
    others_pd = df.loc[df['visitor_location_country_id'] == orig]
    others = others_pd.loc[others_pd['prop_country_id'] == dest]

    if len(others) > 0 and others['orig_destination_distance'].isna().sum() < len(others):
        return np.mean(others['orig_destination_distance'])

    # only from same country
    others = df.loc[df['visitor_location_country_id'] == orig]

    if len(others) > 0 and others['orig_destination_distance'].isna().sum() < len(others):
        return np.mean(others['orig_destination_distance'])

    # only to same country
    others = df.loc[df['prop_country_id'] == dest]

    if len(others) > 0 and others['orig_destination_distance'].isna().sum() < len(others):
        return np.mean(others['orig_destination_distance'])
    
    return 1301

In [9]:
def orig_dest_distance(visitor_country_id, prop_country_id, orig_destination_distance, i):
    orig = visitor_country_id[i]
    dest = prop_country_id[i]

    # check if someone has the exact same but with a distance
    home = np.where(visitor_country_id == orig)
    destination = np.where(prop_country_id == dest)

    others_index = np.intersect1d(home, destination)
    others = np.take(orig_destination_distance, others_index)
    
    nans_np = np.count_nonzero(np.isnan(others))

    if len(others) > 0 and nans_np < len(others):
        return np.nanmean(others)

    # only from same country
    others = np.where(visitor_country_id == orig)
    others = np.take(orig_destination_distance, others)

    nans_np = np.count_nonzero(np.isnan(others))

    if len(others) > 0 and nans_np < len(others):
        return np.nanmean(others)

    # only to same country
    others = np.where(prop_country_id == dest)
    others = np.take(orig_destination_distance, others)

    nans_np = np.count_nonzero(np.isnan(others))
    if len(others) > 0 and nans_np < len(others):
        return np.nanmean(others)
    
    return 1301

In [10]:
def missing_distance(visitor_country_id, prop_country_id, orig_destination_distance):
    for i in tqdm(range(len(orig_destination_distance))):

        if np.isnan(orig_destination_distance[i]):
            result = orig_dest_distance(visitor_country_id, prop_country_id, orig_destination_distance, i)
            orig_destination_distance[i] = result

    return orig_destination_distance

In [11]:
tic()
visitor_country_id = df['visitor_location_country_id'].to_numpy()
prop_country_id = df['prop_country_id'].to_numpy()
orig_destination_distance = df['orig_destination_distance'].to_numpy()

df['orig_destination_distance'] = missing_distance(visitor_country_id, prop_country_id, orig_destination_distance)

if df['orig_destination_distance'].isna().sum() != 0:
    print("distance still has nans!!!!")
toc()

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:19<00:00, 2582.49it/s]

Elapsed time is 19.37813401222229 seconds.


In [12]:
for var in df:
    col = df[var]
    missing = col.isnull().values.sum()
    if missing >0:
        print('the number is missing values in', var, 'is', col.isnull().values.sum())


the number is missing values in visitor_hist_starrating is 47435
the number is missing values in visitor_hist_adr_usd is 47435
the number is missing values in prop_location_score2 is 10892
the number is missing values in srch_query_affinity_score is 46736
the number is missing values in gross_bookings_usd is 48623


# Karim's stuk

In [13]:
import copy
import dateparser
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

from numpy.core.numeric import NaN
from tqdm import tqdm


def trim_cols(data):
    '''
    removes columns with >95% missing data values
    '''
    print('Trimming columns with > 95% missing values:')

    df = data.copy()
    total_values = len(df.index)
    removed_cols=[]
    for col in tqdm(df, total = df.shape[1]):
        missing_values = sum(df[col].isnull()==True)
        if missing_values > .95 * total_values:
            removed_cols.append(col)
            df.drop(col, 1)

    print('Removed cols:')
    for col in removed_cols:
        print(col)
    print('\n')

    return df


def get_daypart(hour):
    ''' 
    retrieves daypart string based on hour of the day
    for trim_dates function
    '''
    if (hour > 6) and (hour <= 12):
        return 'Morning'
    elif (hour > 12) and (hour <= 18 ):
        return 'Afternoon'
    elif (hour > 18) and (hour <= 24):
        return 'Noon'
    else:
        return 'Night'


def trim_dates(dates):
    '''
    replaces datetime column with separate month and daypart column
    '''
    print('Trimming datetime:')

    months, dayparts = [], []
    
    for i in tqdm(range(len(dates))):
        # split date and time
        datetime = dateparser.parse(dates[i])

        # retrieve month 
        month = datetime.date().month
        months.append(month)
                
        # retrieve daypart based on hour of the day
        hour = datetime.time().hour
        daypart = get_daypart(hour)
        dayparts.append(daypart)

    return months, dayparts


def round_halves(raw_values):
    '''
    rounds to halves
    '''
    values = np.nan_to_num(raw_values)
    
    # round ratings to halves, or add NA if data is missing
    for i in tqdm(range(len(values))):
        # progress bar    
        if i % (1/len(values)) == 0:
            print(i/len(values), '%', end='\r')
        
        values[i] = round(values[i] * 2) / 2
    
    return values


def trim_avg_spent(spent):
    '''
    trims average price spent per night for customers

    TODO: kijken of de distributie heel hoog is rondom een getal, en daar misschien meer 'bins' maken
    '''
    print('Trimming visitor_hist_adr_usd:')
    # spent = df['visitor_hist_adr_usd'].to_numpy()
    avg_spent = np.nan_to_num(spent)

    # round to every 50 dollars spent or add 'NA'
    for i in tqdm(range(len(avg_spent))):
        multiplier = round(avg_spent[i]/50)
        avg_spent[i] = multiplier * 50

    return avg_spent


def trim_loc_score(loc_score1, loc_score2):
    '''
    rounds location score to halves or adds 'NA' for missing values
    '''
    print('Trimming prop_location_scores:')
    score1 = np.nan_to_num(loc_score1)
    score2 = np.nan_to_num(loc_score2)

    for i in tqdm(range(len(score1))):
        score1[i] = round(score1[i] * 2) / 2
        score2[i] = round(score2[i] * 2) / 2    

    return score1, score2


def trim_hist_price(price):
    '''
    rounds prices or add NA for values of 0
    '''
    print('Trimming prop_log_historical_price:')
    hist_price = np.nan_to_num(price)
    for i in tqdm(range(len(hist_price))):
        hist_price[i] = round(hist_price[i] * 2) / 2
        
    return hist_price


def trim_price(price):
    print('Trimming price_usd:')
    trimmed_price = np.nan_to_num(price)
    for i in tqdm(range(len(trimmed_price))):    
        trimmed_price[i] = round(trimmed_price[i])

    return trimmed_price
    

def trim_booking_window(bw):
    '''
    changes booking window from days to weeks
    '''
    print('Trimming srch_booking_window')
    booking_window = np.nan_to_num(bw)
    
    for i in tqdm(range(len(booking_window))):
        booking_window[i] = round(booking_window[i]/7)
        
    return booking_window


def trim_dest_dist(bw):
    '''
    divides distance to destination by 100 miles
    '''
    booking_window = np.nan_to_num(bw)
    print('Trimming orig_destination_distance:')

    for i in tqdm(range(len(dest_dist))):
        dest_dist[i] = round(dest_dist[i]/100)
        
    return dest_dist

In [14]:
df

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,...,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,click_bool,gross_bookings_usd,booking_bool,comp_rate,comp_inv,comp_percent
0,0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,...,1,NaN,1301.00,1,0,NaN,0,0.00,0.000000,0.0
1,1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,...,1,NaN,1301.00,1,0,NaN,0,0.00,0.333333,0.0
2,2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,...,1,NaN,1301.00,1,0,NaN,0,0.00,0.000000,0.0
3,3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,...,1,NaN,1301.00,1,0,NaN,0,-0.75,0.250000,5.0
4,4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,...,1,NaN,1301.00,1,0,NaN,0,0.00,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,3314,2013-03-14 12:12:50,5,219,NaN,NaN,18,60256,5,...,1,NaN,886.93,1,0,NaN,0,0.00,0.000000,0.0
49996,49996,3314,2013-03-14 12:12:50,5,219,NaN,NaN,18,67451,4,...,1,-22.4020,899.55,1,0,NaN,0,0.00,0.000000,0.0
49997,49997,3314,2013-03-14 12:12:50,5,219,NaN,NaN,18,71335,4,...,1,NaN,901.27,1,0,NaN,0,0.00,0.000000,0.0
49998,49998,3314,2013-03-14 12:12:50,5,219,NaN,NaN,18,72141,4,...,1,NaN,899.62,1,0,NaN,0,0.00,0.000000,0.0


In [15]:

'''
trims the data, add
_________________________________________________________________
try-excepts have been added in each function to check whether the 
column has not been removed due to >95% of data values missing.
'''

# from trim_functions import *
import time


def trim_data(df):
    # track time
    start_time = time.time()
    
    # trim dates
    dates = df['date_time'].to_numpy()    
    months, dayparts = trim_dates(dates)
    df['month'] = months
    df['daypart'] = dayparts

    # change column order df
    cols = df.columns.tolist()
    cols = cols[1:2] + cols[-2:] + cols[4:-3]
    df = df[cols]
    
    # trim ratings
    print('Trimming visitor_hist_starrating:')
    ratings = df['visitor_hist_starrating'].to_numpy()
    ratings = round_halves(ratings)
    df['visitor_hist_starrating'] = ratings

    # trim avg spent
    spent = df['visitor_hist_adr_usd'].to_numpy()
    avg_spent = trim_avg_spent(spent)
    df['visitor_hist_adr_usd'] = avg_spent

    # trim location scores
    loc_score1 = df['prop_location_score1'].to_numpy()
    loc_score2 = df['prop_location_score2'].to_numpy()
    loc_score1, loc_score2 = trim_loc_score(loc_score1, loc_score2)
    df['prop_location_score1'] = loc_score1
    df['prop_location_score2'] = loc_score2

    # trim hist price
    hist_price = df['prop_log_historical_price'].to_numpy()
    trimmed_hist_price = trim_hist_price(hist_price)
    df['prop_log_historical_price'] = trimmed_hist_price

    # trim price
    price = df['price_usd'].to_numpy()
    trimmed_price = trim_price(price)
    df['price_usd'] = trimmed_price

    # trim booking window
    booking_window = df['srch_booking_window'].to_numpy()
    trimmed_booking_window = trim_booking_window(booking_window)
    df['srch_booking_window'] = trimmed_booking_window

#     @Karim hier errort hij, maar ik denk dat dat komt omdat Daan deze colomn heeft gebruikt
#     # df = trim_dest_dist(df)
#     dest_dist = df['orig_destination_distance'].to_numpy()
#     trimmed_dest_dist = trim_dest_dist(dest_dist)
#     df['orig_destination_distance'] = trimmed_dest_dist
    
    
    print("--- trimming took %s seconds ---" % (round(time.time() - start_time)))
    
    return df

# df = pd.read_csv("../training_set_VU_DM.csv")
# df = pd.read_csv("small_data/3000_training_set_VU_DM.csv")
tic()
data = trim_data(df)
toc()
# data.to_csv('../TRIMMED_training_set_VU_DM.csv')



  0%|                                                                              | 1/50000 [00:00<2:41:13,  5.17it/s]

Trimming datetime:


100%|████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 275461.89it/s]

Trimming visitor_hist_starrating:



<ipython-input-15-895d562f009d>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['visitor_hist_starrating'] = ratings
100%|████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 318218.47it/s]
<ipython-input-15-895d562f009d>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['visitor_hist_adr_usd'] = avg_spent
  0%|                                                                                        | 0/50000 [00:00<?, ?it/s]

...

100%|████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 155093.57it/s]
<ipython-input-15-895d562f009d>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prop_location_score1'] = loc_score1
<ipython-input-15-895d562f009d>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prop_location_score2'] = loc_score2
100%|████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 355560.38it/s]
<ipython-input-15-895d562f00

Trimming prop_log_historical_price:
Trimming price_usd:



<ipython-input-15-895d562f009d>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_usd'] = trimmed_price
100%|████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 404304.56it/s]

Trimming srch_booking_window
--- trimming took 100 seconds ---
Elapsed time is 100.23664784431458 seconds.



<ipython-input-15-895d562f009d>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['srch_booking_window'] = trimmed_booking_window


In [16]:
df

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,...,orig_destination_distance,random_bool,click_bool,gross_bookings_usd,booking_bool,comp_rate,comp_inv,comp_percent,month,daypart
0,0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,...,1301.00,1,0,NaN,0,0.00,0.000000,0.0,4,Morning
1,1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,...,1301.00,1,0,NaN,0,0.00,0.333333,0.0,4,Morning
2,2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,...,1301.00,1,0,NaN,0,0.00,0.000000,0.0,4,Morning
3,3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,...,1301.00,1,0,NaN,0,-0.75,0.250000,5.0,4,Morning
4,4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,...,1301.00,1,0,NaN,0,0.00,0.000000,0.0,4,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,3314,2013-03-14 12:12:50,5,219,NaN,NaN,18,60256,5,...,886.93,1,0,NaN,0,0.00,0.000000,0.0,3,Morning
49996,49996,3314,2013-03-14 12:12:50,5,219,NaN,NaN,18,67451,4,...,899.55,1,0,NaN,0,0.00,0.000000,0.0,3,Morning
49997,49997,3314,2013-03-14 12:12:50,5,219,NaN,NaN,18,71335,4,...,901.27,1,0,NaN,0,0.00,0.000000,0.0,3,Morning
49998,49998,3314,2013-03-14 12:12:50,5,219,NaN,NaN,18,72141,4,...,899.62,1,0,NaN,0,0.00,0.000000,0.0,3,Morning


In [17]:
tic()
# Drop column 'prop_location_score2', and 'gross_bookings_usd' too many missing values.
df = df.drop(columns=['prop_location_score2', 'gross_bookings_usd'])
toc()

Elapsed time is 0.010968446731567383 seconds.


In [18]:
for var in df:
    col = df[var]
    missing = col.isnull().values.sum()
    if missing >0:
        print('the number is missing values in', var, 'is', col.isnull().values.sum())

# Kolommen die we voor nu nog niet hebben aangepast        
# visitor hist starrating
# visitor hist adr usd

the number is missing values in visitor_hist_starrating is 47435
the number is missing values in visitor_hist_adr_usd is 47435
the number is missing values in srch_query_affinity_score is 46736


In [19]:
# Save the processed datafile
df.to_csv('small_data/cleaned_data_50000.csv')